## Convert Ros Bag to Json Files

In [1]:
pip install bagpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys
import json

import rosbag
import numpy as np

from std_msgs.msg import Int32, String

# get the current script's directory
current_directory = os.path.dirname(os.path.abspath(__file__)) if "__file__" in locals() else os.getcwd()
# get the parent directory
parent_directory = os.path.dirname(current_directory)
# Add the parent directory to the sys.path
sys.path.append(parent_directory)

from utils import common

In [3]:
current_path = os.getcwd()
parent_path = os.path.dirname(current_path)
grand_parent_path = os.path.dirname(parent_path)

In [4]:
dataset_path = os.path.join(grand_parent_path, "dataset")
demo_path = os.path.join(dataset_path, "human_demonstrations")

In [5]:
date = "2024_01_23" # year_month_day

In [6]:
ros_topics = ["distance_to_object",
              "distance_to_target",
              "distance_to_start",
              "distance_to_ground",
              "arm_action",
              "arm_state"]

In [7]:
dataset_folder = os.path.join(demo_path, date)

In [8]:
bag_files = os.listdir(dataset_folder + "\\bags")

# Functions

In [9]:
def rosbag_to_json(bag_file,
                   output_json_file):
    
    with rosbag.Bag(bag_file, "r") as bag:
        messages, message_list = [], []

        for topic, msg, t in bag.read_messages():
            
            if topic not in ros_topics:
                continue
            
            if hasattr(msg, "data"):
                message = msg.data
                
                # check if the value is numeric and has at most one decimal point
                if not common.is_float(message):
                    continue
                
            elif hasattr(msg, "position"):
                message = {
                    "position": {
                        "x": msg.position.x,
                        "y": msg.position.y,
                        "z": msg.position.z
                    },
                    "orientation": {
                        "x": msg.orientation.x,
                        "y": msg.orientation.y,
                        "z": msg.orientation.z,
                        "w": msg.orientation.w
                    }
                }
            
            else:
                messages.append(message_list)
                message_list = []
                
                # ignore arm_state for now
                continue
            
            message_dict = {
                "topic": topic,
                "timestamp": str(t),
                "message": message
            }
            message_list.append(message_dict)
    
    with open(output_json_file, "w") as json_file:
        json.dump(messages,
                  json_file,
                  indent=2)

# Test

In [10]:
bag_path = os.path.join(dataset_folder + "\\bags", bag_files[0])

In [11]:
json_path = os.path.join(dataset_folder + "\\jsons", bag_files[0][:bag_files[0].index(".bag")] + ".json")

In [12]:
bag = rosbag.Bag(bag_path)

In [13]:
common.rosbag2json(bag_file=bag_path,
                   output_json_file=json_path,
                   ros_topics=ros_topics)

# Run Through All Files

In [14]:
for file in bag_files:

    bag_path = os.path.join(dataset_folder + "\\bags", file)
    json_path = os.path.join(dataset_folder + "\\jsons", file[:file.index(".bag")] + ".json")
    
    rosbag_to_json(bag_file=bag_path,
               output_json_file=json_path)